In [83]:
import pandas as pd
import requests
import json
import os
import numpy as np
from scipy.stats import poisson
from scipy.stats import norm
from pprint import pprint
import numpy as np
from scipy.optimize import minimize
from scipy.stats import poisson
import warnings
import io


# Suppress divide by zero warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message="divide by zero encountered in log")


API_KEY = os.getenv("API_KEY")
url = 'https://data-service.beatthebookie.blog/data'
headers = {"x-api-key": API_KEY}
params = {'division':'Premier League'}
response = requests.get(url, headers=headers, params=params)
json_str = response.content.decode('utf-8')
prem_df = pd.read_json(io.StringIO(json_str))
prem_teams_25 = prem_df[prem_df["season"] == 20242025]
prem_teams_25 = pd.concat([prem_teams_25['home_team'], prem_teams_25['away_team']]).unique()

params = {'division':'Championship'}
response = requests.get(url, headers=headers, params=params)
json_str = response.content.decode('utf-8')
champ_df = pd.read_json(io.StringIO(json_str))

df = pd.concat([champ_df, prem_df])
#df = df[(df['home_team'].isin(prem_teams_25)) | (df['away_team'].isin(prem_teams_25))]

df['match_date'] = pd.to_datetime(df['match_date'])
df = df[df["match_date"] > '2023-06-01']

print(df[["season", "match_date", "home_team", "away_team", "home_goals", "home_xgoals", "away_goals", "away_xgoals"]].head())

        season match_date       home_team    away_team  home_goals  \
3310  20232024 2023-08-04  Sheffield Weds  Southampton           1   
3311  20232024 2023-08-05    Bristol City      Preston           1   
3312  20232024 2023-08-05         Watford          QPR           4   
3313  20232024 2023-08-05   Middlesbrough     Millwall           0   
3314  20232024 2023-08-05         Norwich         Hull           2   

      home_xgoals  away_goals  away_xgoals  
3310          0.5           2          1.4  
3311          0.9           1          1.3  
3312          2.9           0          0.4  
3313          0.8           1          1.2  
3314          2.6           1          0.4  


In [84]:
# Apply averaged penalty when Championship match contains two promoted teams.

# Penalty xG = 0.665
# Penalty xGA = 1.465

# mask_both_prem = (df['division'] == 'Championship') & (df['home_team'].isin(prem_teams_25)) & (df['away_team'].isin(prem_teams_25))
# # #df.loc[mask_both_prem, 'home_goals'] *= 0.661
# # #df.loc[mask_both_prem, 'away_goals'] *= 0.661
# df.loc[mask_both_prem, 'home_xgoals'] *= 0.665
# df.loc[mask_both_prem, 'away_xgoals'] *= 0.655

# # Adjust performance penalty to championship games when one team is a promoted team.
# mask_home = (df['division'] == 'Championship') & (df['home_team'].isin(prem_teams_25)) & ~(df['away_team'].isin(prem_teams_25))
# # #df.loc[mask_home, 'home_goals'] *= 0.661
# # #df.loc[mask_home, 'away_goals'] *= 2.060
# df.loc[mask_home, 'home_xgoals'] *= 0.665
# df.loc[mask_home, 'away_xgoals'] *= 1.465

# mask_away = (df['division'] == 'Championship') & (df['away_team'].isin(prem_teams_25)) & ~(df['home_team'].isin(prem_teams_25))
# # #df.loc[mask_away, 'home_goals'] *= 2.060
# # #df.loc[mask_away, 'away_goals'] *= 0.661
# df.loc[mask_away, 'home_xgoals'] *= 1.465
# df.loc[mask_away, 'away_xgoals'] *= 0.665


# df[['home_goals', 'away_goals']] = df[['home_goals', 'away_goals']].round()


In [85]:
def decay(xi, t):
    return np.exp(-xi * t)

def rho_correction(goals_home, goals_away, home_exp, away_exp, rho):
    if goals_home == 0 and goals_away == 0:
        return 1 - (home_exp * away_exp * rho)
    elif goals_home == 0 and goals_away == 1:
        return 1 + (home_exp * rho)
    elif goals_home == 1 and goals_away == 0:
        return 1 + (away_exp * rho)
    elif goals_home == 1 and goals_away == 1:
        return 1 - rho
    else:
        return 1.0

In [86]:
def log_likelihood_xg(
    xG_home_observed,
    xG_away_observed,
    home_attack,
    home_defence,
    away_attack,
    away_defence,
    home_advantage,
    rho,
    weight,
    sigma=1.0
):
    xG_expectation_home = np.exp(home_attack + away_defence + home_advantage)
    xG_expectation_away = np.exp(away_attack + home_defence)

    home_llk = norm.pdf(xG_home_observed, loc=xG_expectation_home, scale=sigma)
    away_llk = norm.pdf(xG_away_observed, loc=xG_expectation_away, scale=sigma)

    adj_llk = rho_correction(
        xG_home_observed,
        xG_away_observed,
        xG_expectation_home,
        xG_expectation_away,
        rho,
    )

    if xG_expectation_home < 0 or xG_expectation_away < 0 or adj_llk < 0:
        return 10000

    log_llk = weight * (np.log(home_llk) + np.log(away_llk) + np.log(adj_llk))

    return -log_llk


def fit_xG_model(df, xi=0.0001, sigma=1.0):
    teams = np.sort(np.unique(np.concatenate([df["home_team"], df["away_team"]])))
    n_teams = len(teams)
    
    df["league_strength"] = df["division"].apply(lambda x: 1 if x == "Premier League" else 0.615) 
    df["days_since"] = (df["match_date"].max() - df["match_date"]).dt.days
    df["weight"] = decay(xi, df["days_since"]) * df["league_strength"]

    params = np.concatenate(
        (
            np.random.uniform(0.5, 1.5, (n_teams)),  # attack strength
            np.random.uniform(0, -1, (n_teams)),  # defence strength
            [0.25],  # home advantage
            [-0.1],  # rho
        )
    )

    bounds = [(None, None)] * (2 * n_teams) + [(None, None), (-0.5, 0.5)]

    def _fit(params, df, teams, sigma):
        attack_params = dict(zip(teams, params[:n_teams]))
        defence_params = dict(zip(teams, params[n_teams : (2 * n_teams)]))
        home_advantage = params[-2]
        rho = params[-1]

        llk = list()
        for idx, row in df.iterrows():
            tmp = log_likelihood_xg(
                row["home_xgoals"],
                row["away_xgoals"],
                attack_params[row["home_team"]],
                defence_params[row["home_team"]],
                attack_params[row["away_team"]],
                defence_params[row["away_team"]],
                home_advantage,
                rho,
                row["weight"],
                sigma=sigma
            )
            llk.append(tmp)

        return np.sum(llk)

    options = {
        "maxiter": 100,
        "disp": False,
    }

    constraints = [{"type": "eq", "fun": lambda x: sum(x[:n_teams]) - n_teams}]

    res = minimize(
        _fit,
        params,
        args=(df, teams, sigma),
        constraints=constraints,
        bounds=bounds,
        method='L-BFGS-B',
        options=options,
    )

    model_params = dict(
        zip(
            ["attack_" + team for team in teams]
            + ["defence_" + team for team in teams]
            + ["home_adv", "rho"],
            res["x"],
        )
    )

    # Prepare table data
    attack_values = [model_params[f'attack_{team}'] for team in teams]
    defence_values = [model_params[f'defence_{team}'] for team in teams]
    home_adv = model_params['home_adv']
    rho = model_params['rho']

    # Create DataFrame for attack and defense
    team_strength_df = pd.DataFrame({
        'Team': teams,
        'Attack Strength': attack_values,
        'Defense Strength': defence_values
    })

    # Add home advantage and rho to the DataFrame
    additional_params_df = pd.DataFrame({
        'Parameter': ['Home Advantage', 'Rho'],
        'Value': [home_adv, rho]
    })

    # Print DataFrames
    print("Team Strength (Attack and Defense):")
    print(team_strength_df)
    print("\nAdditional Parameters (Home Advantage and Rho):")
    print(additional_params_df)


    print("Log Likelihood: ", res["fun"])

    return model_params


xg_model_params = fit_xG_model(df, xi=0.001, sigma=1.0)

C:\Users\Owner\AppData\Local\Temp\ipykernel_2096\573921453.py:85: RuntimeWarning: Method L-BFGS-B cannot handle constraints.
  res = minimize(


Team Strength (Attack and Defense):
                Team  Attack Strength  Defense Strength
0            Arsenal         1.481262         -1.523943
1        Aston Villa         1.315310         -0.838541
2         Birmingham         0.421042         -0.442388
3          Blackburn         0.616704         -0.400471
4        Bournemouth         1.290282         -0.844226
5          Brentford         1.267226         -0.936556
6           Brighton         1.239669         -0.952606
7       Bristol City         0.505189         -0.510756
8            Burnley         0.796308         -0.715002
9            Cardiff         0.259359         -0.357179
10           Chelsea         1.485710         -0.887937
11          Coventry         0.615425         -0.570298
12    Crystal Palace         1.068597         -0.930967
13             Derby         0.464041         -0.328371
14           Everton         1.192482         -0.886342
15            Fulham         1.040630         -0.845403
16      Hudd

In [87]:
def log_likelihood_goals(
    goals_home_observed,
    goals_away_observed,
    home_attack,
    home_defence,
    away_attack,
    away_defence,
    home_advantage,
    rho,
    weight
):
    goal_expectation_home = np.exp(home_attack + away_defence + home_advantage)
    goal_expectation_away = np.exp(away_attack + home_defence)

    home_llk = poisson.pmf(goals_home_observed, goal_expectation_home)
    away_llk = poisson.pmf(goals_away_observed, goal_expectation_away)
    adj_llk = rho_correction(
        goals_home_observed,
        goals_away_observed,
        goal_expectation_home,
        goal_expectation_away,
        rho,
    )

    if goal_expectation_home < 0 or goal_expectation_away < 0 or adj_llk < 0:
        return 10000

    log_llk = weight * (np.log(home_llk) + np.log(away_llk) + np.log(adj_llk))

    return -log_llk


def fit_poisson_model(df, xi=0.0001):
    teams = np.sort(np.unique(np.concatenate([df["home_team"], df["away_team"]])))
    n_teams = len(teams)
    
    df["league_strength"] = df["division"].apply(lambda x: 1 if x == "Premier League" else 0.615) 
    df["days_since"] = (df["match_date"].max() - df["match_date"]).dt.days
    df["weight"] = decay(xi, df["days_since"]) * df["league_strength"]

    params = np.concatenate(
        (
            np.random.uniform(0.5, 1.5, (n_teams)),  # attack strength
            np.random.uniform(0, -1, (n_teams)),  # defence strength
            [0.25],  # home advantage
            [-0.1], # rho
        )
    )

    bounds = [(None, None)] * (2 * n_teams) + [(None, None), (-1, 1)]

    def _fit(params, df, teams):
        attack_params = dict(zip(teams, params[:n_teams]))
        defence_params = dict(zip(teams, params[n_teams : (2 * n_teams)]))
        home_advantage = params[-2]
        rho = params[-1]

        llk = list()
        for idx, row in df.iterrows():
            tmp = log_likelihood_goals(
                row["home_goals"],
                row["away_goals"],
                attack_params[row["home_team"]],
                defence_params[row["home_team"]],
                attack_params[row["away_team"]],
                defence_params[row["away_team"]],
                home_advantage,
                rho,
                row["weight"]
            )
            llk.append(tmp)

        return np.sum(llk)

    options = {
        "maxiter": 100,
        "disp": False,
    }

    constraints = [{"type": "eq", "fun": lambda x: sum(x[:n_teams]) - n_teams}]

    res = minimize(
        _fit,
        params,
        args=(df, teams),
        constraints=constraints,
        method='L-BFGS-B',
        options=options,
        bounds=bounds
    )

    model_params = dict(
        zip(
            ["attack_" + team for team in teams]
            + ["defence_" + team for team in teams]
            + ["home_adv", "rho"],
            res["x"],
        )
    )

    # Prepare table data
    attack_values = [model_params[f'attack_{team}'] for team in teams]
    defence_values = [model_params[f'defence_{team}'] for team in teams]
    home_adv = model_params['home_adv']
    rho = model_params['rho']

    # Create DataFrame for attack and defense
    team_strength_df = pd.DataFrame({
        'Team': teams,
        'Attack Strength': attack_values,
        'Defense Strength': defence_values
    })

    # Add home advantage and rho to the DataFrame
    additional_params_df = pd.DataFrame({
        'Parameter': ['Home Advantage', 'Rho'],
        'Value': [home_adv, rho]
    })

    # Print DataFrames
    print("Team Strength (Attack and Defense):")
    print(team_strength_df)
    print("\nAdditional Parameters (Home Advantage and Rho):")
    print(additional_params_df)

    print("Log Likelihood: ", res["fun"])

    return model_params

goals_model_params = fit_poisson_model(df, xi=0.001)

C:\Users\Owner\AppData\Local\Temp\ipykernel_2096\548979547.py:82: RuntimeWarning: Method L-BFGS-B cannot handle constraints.
  res = minimize(


Team Strength (Attack and Defense):
                Team  Attack Strength  Defense Strength
0            Arsenal         1.539223         -1.621080
1        Aston Villa         1.386429         -0.830222
2         Birmingham         0.377464         -0.352022
3          Blackburn         0.626266         -0.244226
4        Bournemouth         1.096062         -0.762177
5          Brentford         1.112108         -0.788375
6           Brighton         1.107523         -0.879512
7       Bristol City         0.471972         -0.518134
8            Burnley         0.861283         -0.621515
9            Cardiff         0.369320         -0.204462
10           Chelsea         1.467364         -0.780069
11          Coventry         0.665186         -0.439936
12    Crystal Palace         1.093527         -0.893514
13             Derby         0.872875         -0.314960
14           Everton         0.726097         -0.906800
15            Fulham         1.056781         -0.859175
16      Hudd

In [88]:
def predict(params, home_team, away_team):
    home_attack = params["attack_" + home_team]
    home_defence = params["defence_" + home_team]
    away_attack = params["attack_" + away_team]
    away_defence = params["defence_" + away_team]
    home_advantage = params["home_adv"]
    #rho = params["rho"] #TODO: Debug RHO on XG model
    rho = -0.13

    home_goal_expectation = np.exp(home_attack + away_defence + home_advantage)
    away_goal_expectation = np.exp(away_attack + home_defence)

    home_probs = poisson.pmf(range(10), home_goal_expectation)
    away_probs = poisson.pmf(range(10), away_goal_expectation)

    m = np.outer(home_probs, away_probs)

    m[0, 0] *= 1 - home_goal_expectation * away_goal_expectation * rho
    m[0, 1] *= 1 + home_goal_expectation * rho
    m[1, 0] *= 1 + away_goal_expectation * rho
    m[1, 1] *= 1 - rho    

    home = np.sum(np.tril(m, -1)) 
    draw = np.sum(np.diag(m)) 
    away = np.sum(np.triu(m, 1))

    total_prob = home + draw + away
    home /= total_prob
    draw /= total_prob
    away /= total_prob 

    # Calculate the probability of a clean sheet for the home team (away team scores 0)
    home_clean_sheet_prob = m[:, 0].sum() 

    # Calculate the probability of a clean sheet for the away team (home team scores 0)
    away_clean_sheet_prob = m[0, :].sum() 

    # Calculate the probability of the home team scoring 3 or more goals
    home_3_plus_goals_prob = home_probs[3:].sum() 

    # Calculate the probability of the away team scoring 3 or more goals
    away_3_plus_goals_prob = away_probs[3:].sum() 


    return {
        "home_win_prob": home.round(2),
        "draw_prob": draw.round(2),
        "away_win_prob": away.round(2),
        "home_clean_sheet_prob": home_clean_sheet_prob.round(2),
        "away_clean_sheet_prob": away_clean_sheet_prob.round(2),
        "home_goal_expectation": home_goal_expectation.round(2),
        "away_goal_expectation": away_goal_expectation.round(2),
        "home_3_plus_goals_prob": home_3_plus_goals_prob.round(2),
        "away_3_plus_goals_prob": away_3_plus_goals_prob.round(2)
    }

In [89]:
# Initialize lists to store predictions
xg_home_preds = []
xg_away_preds = []
goals_home_preds = []
goals_away_preds = []

# Iterate through the games in your dataset
for idx, row in df.iterrows():
    xg_pred = predict(xg_model_params, row['home_team'], row['away_team'])
    goals_pred = predict(goals_model_params, row['home_team'], row['away_team'])

    xg_home_preds.append(xg_pred['home_goal_expectation'])
    xg_away_preds.append(xg_pred['away_goal_expectation'])
    goals_home_preds.append(goals_pred['home_goal_expectation'])
    goals_away_preds.append(goals_pred['away_goal_expectation'])

# Add these predictions back to the DataFrame
df['xg_home_pred'] = xg_home_preds
df['xg_away_pred'] = xg_away_preds
df['goals_home_pred'] = goals_home_preds
df['goals_away_pred'] = goals_away_preds

# Calculate absolute errors for xG predictions
df['xg_home_error'] = abs(df['home_xgoals'] - df['xg_home_pred'])
df['xg_away_error'] = abs(df['away_xgoals'] - df['xg_away_pred'])

# Calculate absolute errors for goals predictions
df['goals_home_error'] = abs(df['home_goals'] - df['goals_home_pred'])
df['goals_away_error'] = abs(df['away_goals'] - df['goals_away_pred'])


# Calculate MAE for xG predictions
xg_mae_home = df['xg_home_error'].mean()
xg_mae_away = df['xg_away_error'].mean()

# Calculate MAE for goals predictions
goals_mae_home = df['goals_home_error'].mean()
goals_mae_away = df['goals_away_error'].mean()

# Combine MAEs for home and away
xg_mae_total = (xg_mae_home + xg_mae_away) / 2
goals_mae_total = (goals_mae_home + goals_mae_away) / 2

print(f"xG MAE (home): {xg_mae_home}")
print(f"xG MAE (away): {xg_mae_away}")
print(f"xG MAE (total): {xg_mae_total}")

print(f"Goals MAE (home): {goals_mae_home}")
print(f"Goals MAE (away): {goals_mae_away}")
print(f"Goals MAE (total): {goals_mae_total}")

xG MAE (home): 0.5693176911177644
xG MAE (away): 0.5215447185628742
xG MAE (total): 0.5454312048403194
Goals MAE (home): 0.9492415169660678
Goals MAE (away): 0.8449500998003993
Goals MAE (total): 0.8970958083832336


In [90]:
fixtures = [
    ("Southampton", "Man United"),
    ("Brighton", "Ipswich"),
    ("Crystal Palace", "Leicester"),
    ("Fulham", "West Ham"),
    ("Liverpool", "Nott'm Forest"),
    ("Man City", "Brentford"),
    ("Aston Villa", "Everton"),
    ("Bournemouth", "Chelsea"),
    ("Tottenham", "Arsenal"),
    ("Wolves", "Newcastle")
]

results = []

for fixture in fixtures:
    home_team, away_team = fixture
    
    xg_preds = predict(xg_model_params, home_team, away_team)
    goals_preds = predict(goals_model_params, home_team, away_team)
    
    weighted_preds = {}
    
    for key in xg_preds.keys():
        weighted_value = (0.7 * xg_preds[key]) + (0.3 * goals_preds[key])
        weighted_preds[key] = weighted_value
    
    results.append({
        'home_team': home_team,
        'away_team': away_team,
        **weighted_preds
    })

results_df = pd.DataFrame(results)


# Display the results DataFrame
display(results_df)

,home_team,away_team,home_win_prob,draw_prob,away_win_prob,home_clean_sheet_prob,away_clean_sheet_prob,home_goal_expectation,away_goal_expectation,home_3_plus_goals_prob,away_3_plus_goals_prob
0,Southampton,Man United,0.307,0.257,0.436,0.171,0.234,1.475,1.783,0.186,0.265
1,Brighton,Ipswich,0.669,0.206,0.125,0.393,0.103,2.300,0.936,0.404,0.072
2,Crystal Palace,Leicester,0.531,0.263,0.206,0.364,0.176,1.733,1.012,0.251,0.083
3,Fulham,West Ham,0.491,0.250,0.266,0.257,0.153,1.886,1.352,0.294,0.153
4,Liverpool,Nott'm Forest,0.718,0.184,0.098,0.434,0.087,2.467,0.843,0.446,0.054
5,Man City,Brentford,0.737,0.175,0.095,0.438,0.081,2.557,0.832,0.470,0.054
6,Aston Villa,Everton,0.526,0.240,0.227,0.291,0.140,1.984,1.264,0.317,0.137
7,Bournemouth,Chelsea,0.366,0.230,0.401,0.144,0.155,1.852,1.936,0.289,0.309
8,Tottenham,Arsenal,0.195,0.247,0.558,0.144,0.345,1.065,1.933,0.091,0.309
9,Wolves,Newcastle,0.254,0.230,0.513,0.113,0.219,1.515,2.163,0.198,0.364


In [91]:
results_df.to_csv("predictions/ensmeble_gw4_preds.csv")